In [ ]:
# https://github.com/microsoft/CameraTraps/blob/main/data_management/lila/create_lila_test_set.py
# https://github.com/cindyweng/coco-to-yolo-by-category/blob/5fcd1ae51af89c1c678d903a4aff5d32cba25b0b/coco-to-yolo-by-category.py#L41

In [2]:
import json
import random
import numpy as np
import shutil
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from operator import itemgetter 
from itertools import groupby
from os.path import exists


random.seed(42)

In [17]:
basepath = 'I:\orinoquia\public\\' # refactor to match path naming
metadata_path = 'I:\orinoquia\orinoquia_camera_traps_metadata\orinoquia_camera_traps.json'
train_path = './data/orinoquia/images/train/'
val_path = './data/orinoquia/images/val/'
test_path = './data/orinoquia/images/test/'
label_path = './data/orinoquia/labels/'

In [18]:
with open(metadata_path) as f:
    d = json.load(f)

In [30]:
# Remove images not found
ne = []
i = 0
for img in d['images']:
    if not exists(basepath + img['file_name']):
        ne.append(img)
        i += 1
print("{} images do not exist (counts may overlap)".format(i))

remove_missing_id = set([n.get('id') for n in ne])

7441 images do not exist (images may overlap)


In [33]:
def gen_dataset(d, remove_missing_id, n_empty=1000, n_nempty=10000):
    n_empty_images_per_dataset = n_empty
    n_non_empty_images_per_dataset = n_nempty

    category_id_to_name = {c['id']:c['name'] for c in d['categories']}
    category_name_to_id = {c['name']:c['id'] for c in d['categories']}


    human_category_id = category_name_to_id['human'] if 'human' in category_name_to_id.keys() else -1 # filter out humans


    if 'empty' not in category_name_to_id:
        print('Warning: no empty images available for {}'.format('dataset'))
        empty_category_id = -1
        empty_annotations = []
        empty_annotations_to_download = []
    else:
        empty_category_id = category_name_to_id['empty']        
        empty_annotations = [ann for ann in d['annotations'] if ann['category_id'] == empty_category_id and ann['image_id'] not in remove_missing_id]
        empty_annotations_to_download = random.sample(empty_annotations, n_empty_images_per_dataset)        
        
    non_empty_annotations = [ann for ann in d['annotations'] if ann['category_id'] not in (empty_category_id, human_category_id) and ann['image_id'] not in remove_missing_id]

    non_empty_annotations_to_download = random.sample(non_empty_annotations, n_non_empty_images_per_dataset)
    annotations_to_download = empty_annotations_to_download + non_empty_annotations_to_download
    image_ids_to_download = set([ann['image_id'] for ann in annotations_to_download])
    assert len(image_ids_to_download) == len(set(image_ids_to_download))

    images_to_download = []
    for im in d['images']:
        if im['id'] in image_ids_to_download:
            images_to_download.append(im)
    assert len(images_to_download) == len(image_ids_to_download)
    
    return images_to_download

In [34]:
images_to_download = gen_dataset(d, remove_missing_id, 1000, 10000)
train, validate, test = np.split(images_to_download, [int(.8*len(images_to_download)), int(.9*len(images_to_download))])

In [35]:
trlabelpath, vlabelpath, telabelpath = label_path + "train/", label_path + "val/", label_path + 'test/'

for p in (train_path, val_path, test_path, trlabelpath, vlabelpath, telabelpath):
    files = glob.glob(p)
    for f in glob.glob(p + '\*', recursive=True):
        if f.endswith('.jpg') or f.endswith('.txt'):
            os.remove(f)

for im in train: 
    newfile = shutil.copy2(basepath+im['file_name'], train_path)
    newname = os.path.dirname(newfile) + "/" + im['file_name'].replace("/", "-") # name should match label - multiple 001, 002 files...
    os.rename(newfile, newname)
for im in validate: 
    newfile = shutil.copy2(basepath+im['file_name'], val_path)
    newname = os.path.dirname(newfile) + "/" + im['file_name'].replace("/", "-") # name should match label - multiple 001, 002 files...
    os.rename(newfile, newname)
for im in test: 
        newfile = shutil.copy2(basepath+im['file_name'], test_path)
        newname = os.path.dirname(newfile) + "/" + im['file_name'].replace("/", "-") # name should match label - multiple 001, 002 files...
        os.rename(newfile, newname)

In [39]:
d['categories']

[{'name': 'empty', 'id': 0, 'count': 20334},
 {'id': 1, 'name': 'human', 'count': 7441},
 {'id': 2, 'name': 'black_agouti', 'count': 14206},
 {'id': 3, 'name': 'collared_peccary', 'count': 24784},
 {'id': 4, 'name': 'spixs_guan', 'count': 1317},
 {'id': 5, 'name': 'rodent', 'count': 4135},
 {'id': 6, 'name': 'unknown_bird', 'count': 5766},
 {'id': 7, 'name': 'unknown', 'count': 1280},
 {'id': 8, 'name': 'giant_anteater', 'count': 807},
 {'id': 9, 'name': 'ornate_titi_monkey', 'count': 49},
 {'id': 10, 'name': 'lowland_tapir', 'count': 1562},
 {'id': 11, 'name': 'unknown_armadillo', 'count': 5732},
 {'id': 12, 'name': 'unknown_squirrel_monkey', 'count': 50},
 {'id': 13, 'name': 'puma', 'count': 131},
 {'id': 14, 'name': 'southern_tamandua', 'count': 1391},
 {'id': 15, 'name': 'cattle', 'count': 3580},
 {'id': 16, 'name': 'south_american_coati', 'count': 2895},
 {'id': 17, 'name': 'ants', 'count': 33},
 {'id': 18, 'name': 'amazonian_motmot', 'count': 268},
 {'id': 19, 'name': 'unknown_po

In [42]:

def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier


def createLabelsSingle(imageList, basedir, labeldirname, metadata_full):
    # For single objects only

    ids = [i.get('id') for i in imageList]
    # generate lookup for bbox and category id based on image id


    print("!WARNING: hardcoded fix for islands dataset")

    lookup = {}
    for meta in metadata_full["annotations"]:
        if meta["image_id"] not in ids: continue

        bb = [0, 0, 1919, 1079] #TODO this is hardcoded fix/default for the islands dataset 

        try:
            bb = meta['bbox']
        except KeyError:
            if meta['category_id'] != 0:
                raise KeyError('Keyerror on boundingbox but not an empty image! meta ID: {}'.format(meta))

        lookup[meta['image_id']] = {"bbox": bb, "category_id": meta["category_id"]}


    for im in imageList:

        ann = lookup.get(im['id'])

        dw = 1. / im['width']
        dh = 1. / im['height']
        
        
        filename = im['file_name'].replace(".jpg", ".txt").replace("/", "-")
        # print(Path(basedir).parent.__str__() + "/labels/" + labeldirname + filename, "a")
        with open(Path(basedir).parent.parent.__str__() + "/labels/" + labeldirname + filename, "a") as myfile:
            xmin = ann["bbox"][0]
            ymin = ann["bbox"][1]
            xmax = ann["bbox"][2] + ann["bbox"][0]
            ymax = ann["bbox"][3] + ann["bbox"][1]
            
            x = (xmin + xmax)/2
            y = (ymin + ymax)/2
            
            w = xmax - xmin
            h = ymax-ymin
            
            x = x * dw
            w = w * dw
            y = y * dh
            h = h * dh
            
            mystring = str(str(ann['category_id']) + " " + str(truncate(x, 7)) + " " + str(truncate(y, 7)) + " " + str(truncate(w, 7)) + " " + str(truncate(h, 7)))
            myfile.write(mystring)
            myfile.write("\n")

        myfile.close()

In [43]:
createLabelsSingle(train, train_path, 'train/', d)
createLabelsSingle(validate, val_path, 'val/', d)
createLabelsSingle(test, test_path, 'test/', d)

# sanity check: 
assert((len(train)+ len(validate) + len(test)) == (len(glob.glob(label_path + "/test/*")) + len(glob.glob(label_path + "/train/*")) + len(glob.glob(label_path + "/val/*"))))

# TODO perpaps add labels?
with open(Path(label_path).parent.__str__() + '/description', 'w') as f:
    f.write('train, val, test,\n{}, {}, {}'.format(len(train), len(validate), len(test)))

!WARNING: hardcoded fix for islands dataset


KeyError: "Keyerror on boundingbox but not an empty image! meta ID: {'id': '3a6e598a-7160-11ec-b9b4-5cf3706028c2', 'image_id': 'A01_01100087.JPG', 'category_id': 2, 'sequence_level_annotation': False}"

In [ ]:
# TODO auto generate YAML
# import yaml

# data = dict('path' : './data/islands',  # dataset root dir
#     'train' : 'images/train2017',  # train images (relative to 'path') 128 images
#     'val' : 'images/train2017',  # val images (relative to 'path') 128 images
#     A = 'a',
#     B = dict(
#         C = 'c',
#         D = 'd',
#         E = 'e',
#     )
# )


# with open('data.yml', 'w') as outfile:
#     yaml.dump(data, outfile, default_flow_style=False)